# Stocking rental bikes

![bike rentals](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Bay_Area_Bike_Share_launch_in_San_Jose_CA.jpg/640px-Bay_Area_Bike_Share_launch_in_San_Jose_CA.jpg)

You stock bikes for a bike rental company in Austin, ensuring stations have enough bikes for all their riders. You decide to build a model to predict how many riders will start from each station during each hour, capturing patterns in seasonality, time of day, day of the week, etc.

To get started, create a project in GCP and connect to it by running the code cell below. Make sure you have connected the kernel to your GCP account in Settings.

In [1]:
# Set your own project id here
PROJECT_ID = 'organic-duality-251417' # a string, like 'kaggle-bigquery-240818'

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location="US")
dataset = client.create_dataset('model_dataset', exists_ok=True)

from google.cloud.bigquery import magics
from kaggle.gcp import KaggleKernelCredentials
magics.context.credentials = KaggleKernelCredentials()
magics.context.project = PROJECT_ID

In [2]:
%load_ext google.cloud.bigquery

## Linear Regression

Your dataset is quite large. BigQuery is especially efficient with large datasets, so you'll use BigQuery-ML (called BQML) to build your model. BQML uses a "linear regression" model when predicting numeric outcomes, like the number of riders.

## 1) Training vs testing

You'll want to test your model on data it hasn't seen before.

## Training data

First, you'll write a query to get the data for model-building. You can use the public Austin bike share dataset from the `bigquery-public-data.austin_bikeshare.bikeshare_trips` table. You predict the number of rides based on the station where the trip starts and the hour when the trip started. Use the `TIMESTAMP_TRUNC` function to truncate the start time to the hour.

## 2) Exercise: Query the training data

Write the query to retrieve your training data. The fields should be:
1. The start_station_name
2. A time trips start, to the nearest hour. Get this with `TIMESTAMP_TRUNC(start_time, HOUR) as start_hour`
3. The number of rides starting at the station during the hour. Call this `num_rides`.
Select only the data before 2018-01-01 (so we can save data from 2018 as testing data.)

In [3]:
# create a reference to our table
table = client.get_table("bigquery-public-data.austin_bikeshare.bikeshare_trips")

# look at five rows from our dataset
client.list_rows(table, max_results=5).to_dataframe()

trip_id                     subscriber_type bikeid  \
0  9900285908  Annual Membership (Austin B-cycle)    400   
1  9900289692                             Walk Up    248   
2  9900285987      24-Hour Kiosk (Austin B-cycle)    446   
3  9900285989      24-Hour Kiosk (Austin B-cycle)    203   
4  9900285991      24-Hour Kiosk (Austin B-cycle)    101   

                 start_time  start_station_id  \
0 2014-10-26 14:12:00+00:00              2823   
1 2015-10-02 21:12:01+00:00              1006   
2 2014-10-26 15:12:00+00:00              2712   
3 2014-10-26 15:12:00+00:00              2712   
4 2014-10-26 15:12:00+00:00              2712   

                        start_station_name end_station_id  \
0  Capital Metro HQ - East 5th at Broadway           2544   
1                         Zilker Park West           1008   
2                  Toomey Rd @ South Lamar           2712   
3                  Toomey Rd @ South Lamar           2712   
4                  Toomey Rd @ South Lamar           2712   

            end_station_name  duration_minutes  
0  East 6th & Pedernales St.                10  
1               Nueces @ 3rd                39  
2    Toomey Rd @ South Lamar                31  
3    Toomey Rd @ South Lamar                31  
4    Toomey Rd @ South Lamar                30

Write your query below:

In [4]:
%%bigquery train_data
SELECT start_station_name, 
       TIMESTAMP_TRUNC(start_time, HOUR) as start_hour,
       COUNT(bikeid) as num_rides
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_time < "2018-01-01"
GROUP BY start_station_name, start_hour

In [5]:
train_data.head(10)

start_station_name                start_hour  num_rides
0   Zilker Park West 2015-10-02 21:00:00+00:00         88
1   Zilker Park West 2015-10-02 20:00:00+00:00         23
2   Zilker Park West 2015-10-03 21:00:00+00:00         78
3       Nueces @ 3rd 2015-10-03 12:00:00+00:00         11
4   Zilker Park West 2015-10-03 16:00:00+00:00         12
5   Zilker Park West 2015-10-03 20:00:00+00:00         26
6   Zilker Park West 2015-10-03 18:00:00+00:00         11
7   Zilker Park West 2015-10-03 17:00:00+00:00          9
8   Zilker Park West 2015-10-03 19:00:00+00:00         22
9  Rainey @ River St 2015-10-04 14:00:00+00:00         11

## Model creation

Now it's time to turn this data into a model. You'll use the `CREATE MODEL` statement that has a structure like: 

```sql
CREATE OR REPLACE MODEL`model_dataset.bike_trips`
OPTIONS(model_type='linear_reg') AS 
-- training data query goes here
SELECT ...
    column_with_labels AS label
    column_with_data_1 
    column_with_data_2
FROM ... 
WHERE ... (Optional)
GROUP BY ... (Optional)
```

The `model_type` and `optimize_strategy` shown here are good parameters to use in general for predicting numeric outcomes with BQML.

**Tip:** Using ```CREATE OR REPLACE MODEL``` rather than just ```CREATE MODEL``` ensures you don't get an error if you want to run this command again without first deleting the model you've created.

## 3) Exercise: Create and train the model

Below, write your query to create and train a linear regression model on the training data.

Write your query below:

In [6]:
%%bigquery
CREATE OR REPLACE MODEL `model_dataset.bike_trips`
OPTIONS(model_type='linear_reg') AS
SELECT COUNT(bikeid) as label, 
       start_station_name, 
       TIMESTAMP_TRUNC(start_time, HOUR) as start_hour
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_time < "2018-01-01"
GROUP BY start_station_name, start_hour

Empty DataFrame
Columns: []
Index: []

## 4) Exercise: Model evaluation

Now that you have a model, evaluate it's performance on data from 2018. 


> Note that the ML.EVALUATE function will return different metrics depending on what's appropriate for your specific model. You can just use the regular ML.EVALUATE funciton here. (ROC curves are generally used to evaluate binary problems, not linear regression, so there's no reason to plot one here.)

Write your query below:

In [7]:
%%bigquery
SELECT *
FROM
ML.EVALUATE(MODEL `model_dataset.bike_trips`, (
SELECT COUNT(bikeid) as label, 
       start_station_name, 
       TIMESTAMP_TRUNC(start_time, HOUR) as start_hour
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_time >= "2018-01-01"
GROUP BY start_station_name, start_hour
))

mean_absolute_error  mean_squared_error  mean_squared_log_error  \
0            19.887976          726.444972                4.539111   

   median_absolute_error   r2_score  explained_variance  
0               6.989224 -65.861235          -29.460766

You should see that the r^2 score here is negative. Negative values indicate that the model is worse than just predicting the mean rides for each example.

## 5) Theories for poor performance

Why would your model be doing worse than making the most simple prediction based on historical data?

In [8]:
## Thought question answer here
# Something has changed that we didn't account for.

## 6) Exercise: Looking at predictions

A good way to figure out where your model is going wrong is to look closer at a small set of predictions. Use your model to predict the number of rides for the 22nd & Pearl station in 2018. Compare the mean values of predicted vs actual riders.

Write your query below:

In [9]:
%%bigquery

SELECT AVG(ROUND(predicted_label)) as predicted_avg_riders, 
       AVG(label) as true_avg_riders
FROM
ML.PREDICT(MODEL `model_dataset.bike_trips`, (
SELECT COUNT(bikeid) as label,
       start_station_name,
       TIMESTAMP_TRUNC(start_time, HOUR) as start_hour
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_time >= "2018-01-01" AND start_time < "2019-01-01"
  AND start_station_name = "22nd & Pearl"
GROUP BY start_station_name, start_hour
))

predicted_avg_riders  true_avg_riders
0             -34.48249         3.954864

What you should see here is that the model is underestimating the number of rides by quite a bit. 

## 7) Exercise: Average daily rides per station

Either something is wrong with the model or something surprising is happening in the 2018 data. 

What could be happening in the data? Write a query to get the average number of riders per station for each year in the dataset and order by the year so you can see the trend. You can use the `EXTRACT` method to get the day and year from the start time timestamp.

Write your query below:

In [10]:
%%bigquery
WITH daily_rides AS (
    SELECT COUNT(bikeid) AS num_rides,
           start_station_name,
           EXTRACT(DAYOFYEAR from start_time) AS doy,
           EXTRACT(YEAR from start_time) AS year
    FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
    GROUP BY start_station_name, doy, year
    ORDER BY year
), 
station_averages AS (
    SELECT avg(num_rides) AS avg_riders, start_station_name, year
    FROM daily_rides
    GROUP BY start_station_name, year)

SELECT avg(avg_riders) AS daily_rides_per_station, year
FROM station_averages
GROUP BY year
ORDER BY year

daily_rides_per_station  year
0                12.115702  2013
1                10.360497  2014
2                12.495171  2015
3                12.459424  2016
4                 9.768301  2017
5                15.194780  2018
6                 5.065832  2019

## 8) What do your results tell you?

Given the daily average riders per station over the years, does it make sense that the model is failing?

In [11]:
## Thought question answer here
# Yes; the number of riders per station was much higher in 2018 than 2019. 